## excercise 1

In [2]:
import pandas as pd
df = pd.read_excel('dataProject4.xlsx',header=4,sheet_name='20000-211000')
df=df.iloc[:,:7].join(df.iloc[:,7:].rename(columns={df.iloc[:,7:].columns[i]:f'volume {2018+i}' for i in range(len(df.iloc[:,7:].columns))}))

In [25]:
df[df['Postal Code'].str.len()!=7]
#fucked up postal code havers

,2f. Customer number,Postal Code,Customer Classification (CRM),Horeca menu webshop,Prd. name Webshop,1c. Brand name,Contents,volume 2018,volume 2019,volume 2020,volume 2021,volume 2022
72909,210055,52538,Drankenhandel,Bier - Buitenlands P,Bier,FAHNENBRAU,0.5,658.8,216.0,0.0,0.0,0.0
72910,210055,52538,Drankenhandel,Bier - Pils(achtige),Bier,FAHNENBRAU,0.5,0.0,108.0,205.2,162.0,129.6
72911,210055,52538,Result,NaN,NaN,NaN,NaN,658.8,324.0,205.2,162.0,129.6


In [26]:
df

,2f. Customer number,Postal Code,Customer Classification (CRM),Horeca menu webshop,Prd. name Webshop,1c. Brand name,Contents,volume 2018,volume 2019,volume 2020,volume 2021,volume 2022
0,200001,5735 SC,Voetbal,Bier - Alcoholvrij/a,0.0,PALM,0.25,0.000,0.000,0.060,0.000,0.0000
1,200001,5735 SC,Voetbal,Bier - Alcoholvrij/a,0.0 IPA,BAVARIA,0.30,0.000,0.072,0.144,0.000,0.0000
2,200001,5735 SC,Voetbal,Bier - Alcoholvrij/a,0.0 Original,BAVARIA,0.30,0.000,0.000,0.000,0.072,0.4320
3,200001,5735 SC,Voetbal,Bier - Alcoholvrij/a,0.0 Radler,BAVARIA,0.30,0.000,0.792,0.360,0.180,0.4320
4,200001,5735 SC,Voetbal,Bier - Alcoholvrij/a,0.0 Wit,BAVARIA,0.30,0.000,0.000,0.000,0.000,0.0864
...,...,...,...,...,...,...,...,...,...,...,...,...
82329,211000,6815 AH,Jeugdherberg,Wijn - Wit,Winzerschoppen,KONIGSHOF,1.00,0.000,0.540,0.120,0.060,0.4320
82330,211000,6815 AH,Jeugdherberg,Zuivel - Chocomel,Not assigned,CHOCOMEL,0.20,0.960,0.768,0.528,0.720,0.5184
82331,211000,6815 AH,Jeugdherberg,Zuivel - Fruitzuivel,Rood Fruit,FRISTI,0.20,0.336,0.288,0.240,0.240,0.7488
82332,211000,6815 AH,Jeugdherberg,Not assigned,Not assigned,BAVARIA,0.30,0.000,0.000,0.072,0.000,0.0864


In [3]:
print(f'are there any NaN cells in non-result rows? {(df[df.isna().any(axis=1)]['Customer Classification (CRM)']!='Result').any()}')

are there any NaN cells in non-result rows? False


## excercise 2

In [23]:
import pandas as pd
df=pd.read_excel('hotelBookings.xlsx')

In [24]:
#all items in the hotel column are resort hotel and all items in the company column are NaN, therefore these will be removed
removed_columns=['hotel','company']
df=df[[item for item in df.columns if item not in removed_columns]]

## excersice 3

In [25]:
import pandas as pd
df=pd.read_excel('detailedRetail.xlsx')

In [26]:
df

,Category,Month,Sales,Sales Manager
0,"Retail and food services sales, total",January,529616,Donald Ducker
1,Retail sales and food services excl motor vehi...,January,423982,Dominique Kai
2,Retail sales and food services excl gasoline s...,January,487059,Dominique Kai
3,Retail sales and food services excl motor vehi...,January,381425,Dominique Kai
4,"Retail sales, total",January,464062,Jane Maria
...,...,...,...,...
175,Miscellaneous stores retailers,May,8837,Vicky Dullo
176,Nonstore retailers,May,84845,John Doe
177,Electronic shopping and mail order houses,May,78974,Martin Miller
178,Fuel dealers,May,2017,Martin Miller


## excercise 4

In [27]:
# pip install langdetect

In [28]:
import pandas as pd
import langdetect as ld
df=pd.read_excel('tweets.xlsx')

In [29]:
df

,Tweet ID,Country,Date,Tweet,Definitely English,Ambiguous,Definitely Not English,Code-Switched,Ambiguous due to Named Entities,Automatically Generated Tweets
0,434215992731136000,TR,2014-02-14,BugÃ¼n bulusmami lazimdiii,0,0,1,0,0,0
1,285903159434563008,TR,2013-01-01,Volkan konak adami tribe sokar yemin ederim :D,0,0,1,0,0,0
2,285948076496142016,NL,2013-01-01,Bed,1,0,0,0,0,0
3,285965965118824000,US,2013-01-01,I felt my first flash of violence at some fool...,1,0,0,0,0,0
4,286057979831275008,US,2013-01-01,Ladies drink and get in free till 10:30,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
10497,774941788247298048,TR,2016-09-11,"I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...",1,0,0,0,0,1
10498,774951242422480000,PH,2016-09-11,"El Nido, fica ao norte de Palawan, uma das ilh...",0,0,1,0,0,0
10499,774960083721531008,ID,2016-09-11,Alhamdulillah ðŸ˜ŠðŸ‘¨â€ðŸ‘©â€ðŸ‘§â€ðŸ‘§ðŸŽ...,0,0,1,0,0,0
10500,775057244798849024,NG,2016-09-11,Eid-Mubarak @ Bauchi Fedral Lowcost https://t....,0,1,0,0,1,0


In [38]:
#ld.detect is very unreliable :(
tweets_df = pd.DataFrame(df['Tweet'],columns = ['Tweet'])

def detect_language(tweet):

    """
    function to detect a language from tweets dataframe

    args -> tweet: string
        
    returns -> language of tweet
    """
    
    #seed to make avoid fluctuations in the prediction
    ld.DetectorFactory.seed = 20

    #ignore row if it's not a string
    if not isinstance(tweet, str):
        return 'Unknown'
    
    #use langdetect to predict the languages in the tweet
    try:
        return ld.detect(tweet) #or ld.detect_langs(tweet) for more predictions
    except ld.LangDetectException:
        return 'Unknown'

tweets_df['Language'] = 'Unknown'
tweets_df.loc[df['Definitely English'] == 1, 'Language'] = 'en'

mask = tweets_df['Language'] == 'Unknown'
tweets_df.loc[mask, 'Language'] = tweets_df.loc[mask, 'Tweet'].apply(detect_language)

tweets_df

,Tweet,Language
0,BugÃ¼n bulusmami lazimdiii,tr
1,Volkan konak adami tribe sokar yemin ederim :D,tr
2,Bed,en
3,I felt my first flash of violence at some fool...,en
4,Ladies drink and get in free till 10:30,en
...,...,...
10497,"I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...",en
10498,"El Nido, fica ao norte de Palawan, uma das ilh...",pt
10499,Alhamdulillah ðŸ˜ŠðŸ‘¨â€ðŸ‘©â€ðŸ‘§â€ðŸ‘§ðŸŽ...,id
10500,Eid-Mubarak @ Bauchi Fedral Lowcost https://t....,de


In [31]:
test_tweet = ['my name is Lotus', 'mein Name ist Lotus', 'mijn naam is Lotus']
test_df = pd.DataFrame(test_tweet, columns=['Tweet'])

test_df['language'] = test_df['Tweet'].apply(detect_language)
test_df

#shorter texts are going to be predicted wrongly

,Tweet,language
0,my name is Lotus,en
1,mein Name ist Lotus,de
2,mijn naam is Lotus,nl
